In [1]:
import mercury as mr
app = mr.App(title="Практическая работа", description="Расчет дебита скважины")
h = mr.Text(value="20", label="Высота (м):")
fi = mr.Text(value ="0.15", label="Пористость, (ед)")
k = mr.Text(value="1e-15", label="Проницаемость, м^2")
mu = mr.Text(value="1e-3", label="Динамическая вязкость, Па·с")
ro = mr.Text(value="1000", label="Плотность, кг/м^3")
c = mr.Text(value="0.5e-9", label="Сжимаемость, Па^-1")
r_w = mr.Text(value="0.12", label="Радиус скважины, м")
p_0 = mr.Text(value="250e5", label="Начальное давление, Па")
p_zab = mr.Text(value="150e5", label="Забойное давление, Па")

mercury.App

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expi


In [3]:
class wellcalculator:
    def __init__(self):
        self.dt = 86400
        self.N = 300

    def calculate_flow(self, h, fi, k, mu, c, ro,  p_0, r_w, p_zab):      
        kf = k / (fi * mu * c)
        B = - (r_w**2) / (4 * kf * self.dt)
        Ei0 = -expi(B)
        A = 4 * kf * np.pi * fi * c * h / Ei0
        kV = np.arange(1, self.N + 1)
        tV = kV * self.dt
        Q = np.zeros(self.N)
        
        Ei = np.zeros(self.N)
        for n in range(self.N):
            current_k = n + 1
            Ei[n] = -expi(B/current_k) - (-expi(B/(current_k + 1)))

        E = Ei / Ei0
        
        Q[0] = A * (p_0 - pressure_profile[0]) 

        for n in range (1, self.N):
            ii = np.arange(0, n)
            ii_rev = ii[::-1]
            sum_term = np.dot(E[ii], Q[ii_rev])
            Q[n] = A * (p_0 - pressure_profile[n]) + sum_term

        return tV, Q, kf

In [4]:
calculator = wellcalculator()

hr = float(h.value)
fir = float(fi.value) 
kr = float(k.value)
mur = float(mu.value)
cr = float(c.value)
ror = float(ro.value)
p_0r = float(p_0.value)
p_zabr = float(p_zab.value)
p_injr = p_0r
r_wr = float(r_w.value)

pressure_profile = np.ones(calculator.N) * p_zabr
pressure_profile[0:50] = p_injr
pressure_profile[50:] = p_zabr

time_seconds, flow_rate, kf = calculator.calculate_flow(hr, fir, kr, mur, cr,ror, p_0r, r_wr, pressure_profile)
time_days = time_seconds / 86400
flow_rate_m3_day = flow_rate * 86400

mr.Markdown("""
## Основные результаты расчета

| Параметр | Значение | Единицы измерения |
|----------|----------|------------------|
| Коэффициент пьезопроводности | {kf:.2e} | м²/с |
| Максимальный дебит | {max_q:.2f} | м³/сут |
| Средний дебит | {mean_q:.2f} | м³/сут |
| Перепад давления | {delta_p:.1f} | бар |
| Время | {time:.0f} | день |
""".format(
    kf=kf,
    max_q=flow_rate_m3_day.max(), 
    mean_q=flow_rate_m3_day.mean(),
    delta_p=(p_0r - p_zabr) / 1e5,
    time = time_days[-1],
))


## Основные результаты расчета

| Параметр | Значение | Единицы измерения |
|----------|----------|------------------|
| Коэффициент пьезопроводности | 1.33e-02 | м²/с |
| Максимальный дебит | 17.95 | м³/сут |
| Средний дебит | 10.85 | м³/сут |
| Перепад давления | 100.0 | бар |
| Время | 300 | день |


In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
ax1.plot(time_days, np.ones(calculator.N) * p_0r / 1e5, '--b', label='Пластовое давление')
ax1.plot(time_days, pressure_profile / 1e5, '-r', label='Забойное давление')
ax1.set_ylabel('Давление, бар')
ax1.set_ylim([0,500])
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Профиль давления в скважине')
    
# ax2.plot(time_days, flow_rate_m3_day, '-g')
ax2.plot(time_days[:50], flow_rate_m3_day[:50], '-g', label='До скачка')
ax2.plot(time_days[49:52], flow_rate_m3_day[49:52], '--g', linewidth=2, label='Скачок')
ax2.plot(time_days[51:], flow_rate_m3_day[51:], '-g', label='После скачка')
ax2.set_ylabel('Дебит, м³/сут')
ax2.set_xlabel('Время, дни')
ax2.grid(True, alpha=0.3)
ax2.set_title('Дебит скважины во времении')
ax2.legend()

ax1.set_xlim([0, time_days[-1]])
ax2.set_xlim([0, time_days[-1]])

plt.tight_layout()


In [ ]:
import io
import base64
from matplotlib.backends.backend_pdf import PdfPages

pdf_buffer = io.BytesIO()
with PdfPages(pdf_buffer) as pdf:
    pdf.savefig(fig, bbox_inches='tight')
pdf_buffer.seek(0)

pdf_b64 = base64.b64encode(pdf_buffer.getvalue()).decode()
pdf_download = f'<a href="data:application/pdf;base64,{pdf_b64}" download="результаты_расчета.pdf">Скачать PDF с графиками</a>'
mr.Markdown(pdf_download)